In [81]:
%load_ext autoreload
%autoreload 2

import random

import numpy as np
import pandas as pd
from IPython.display import clear_output
from tqdm.auto import tqdm

from modules.data_manage.load_mnist import load_mnist
from modules.grammar.count_accuracy import CountAccuracy
from modules.grammar.grammar import Grammar
from modules.grammar.merge.stupid_merge_same_nodes import stupid_merge_same_nodes
from modules.grammar.node import merge_nodes
from modules.plots.draw_imgs import draw_imgs
from modules.plots.plot_random_from_grammar import plot_random_from_grammar
from modules.utils import inject_jupyter_style

inject_jupyter_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
x_train, y_train, x_test, y_test = load_mnist(size=10000)
# draw_imgs(x_train, (2, 7), random=False)

Train: (10000, 32, 32) (10000,)
Test:  (10000, 32, 32) (10000,)


In [ ]:
grammar = Grammar(colors=x_train.max() + 1)
metric = CountAccuracy(
    grammar=grammar,
    train=x_train[y_train == 0],
    valid=x_test[y_test == 0],
    wrong_train=x_train[y_train == 1],  # [x_train[y_train == i] for i in range(1, 10)],
    wrong_valid=x_test[y_test == 1],  # [x_test[y_test == i] for i in range(1, 10)],
)
grammar.add_images(x_train[y_train == 0])

stupid_merge_same_nodes(grammar)
nodes, node_df = grammar.set_height_for_all_nodes(verbose=False)
res = metric.count_metrics(verbose=True)
res

  0%|          | 0/1001 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df = pd.concat(
    [node_df.reset_index().drop(columns="index"), pd.DataFrame([res])], axis=1
)
display(df)

for _ in range(10):
    print("merging....")
    for layer in range(1, 9):
        if len(nodes[layer]) > 1:
            random.shuffle(nodes[layer])
            for i in range(0, len(nodes[layer]) - 1, 2):
                merge_nodes(nodes[layer][i], nodes[layer][i + 1])
    print("stupid_merge_same_nodes")
    stupid_merge_same_nodes(grammar)
    print("set_height_for_all_nodes")
    nodes, node_df = grammar.set_height_for_all_nodes(verbose=False)

    print("count_metrics")
    tmp_df = pd.DataFrame([metric.count_metrics(verbose=True)])
    tmp_df = pd.concat([tmp_df, node_df.reset_index().drop(columns="index")], axis=1)
    df = pd.concat([df, tmp_df])

    clear_output(wait=True)

    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,train,valid,wrong train,wrong valid
0,2,4,24,84,143,181,127,80,50,26,13,1,1.0,0.0,0.0,0.0
0,2,2,12,42,72,91,64,40,25,26,13,1,1.0,0.0,0.0,0.0
0,2,1,6,21,36,46,32,20,13,26,13,1,1.0,0.0,0.0,0.0


merging....
stupid_merge_same_nodes
set_height_for_all_nodes
count_metrics


  0%|          | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 